# INTACT pipeline
run intact on the ptwas and fastenloc output

## Example
We now run an example based on [example fastenloc and ptwas output from Tiffany, Montgomery lab]( https://github.com/tyeulalio/QTL_pipeline)


In [ ]:
sos run pipeline/intact.ipynb intact \
    --fastenloc_file /mnt/vast/hpc/csg/rf2872/data/INTACT_test/04_colocalization/01_run_fastenloc/fastqtl_.enloc.gene.out \
    --ptwas_file /mnt/vast/hpc/csg/rf2872/data/INTACT_test/05_ptwas/04_ptwas_scan/all_chroms_ptwas_scan.stratified_out.txt \
    --tissue DLPFC

In [ ]:
[global]
# Workdir
parameter: cwd = path("output")
# fastenloc output
parameter: fastenloc_file = ""
# ptwas output
parameter: ptwas_file = ""
# dataset 
parameter: tissue = ''
# QTL data type
parameter: QTL = 'eQTL'
parameter: container = ''
parameter: entrypoint={('micromamba run -n' + ' ' + container.split('/')[-1][:-4]) if container.endswith('.sif') else f''}
parameter: job_size = 1
parameter: walltime = "5h"
parameter: mem = "8G"
parameter: numThreads = 1


In [ ]:
[intact]
input: ptwas_file, fastenloc_file
output: f'{cwd}/{tissue}.INTACT.rds'
task: trunk_workers = 1, trunk_size = job_size, walltime = walltime, mem = mem, cores = numThreads, tags = f'{step_name}_{_output:bn}'
R: expand= "${ }", stderr = f'{_output:nn}.stderr', stdout = f'{_output:nn}.stdout'
    library(INTACT)
    #library(biomaRt)
    library(tidyverse)

    # run intact on the ptwas and colocalization output


    run_intact <- function(ptwas, fastenloc){
        # run intact
        head(ptwas) 
        head(fastenloc)

        summary(ptwas$stat)

        # join the columns that we want
        sub_ptwas <- ptwas %>%
            select(gene, zscore=stat)
        head(sub_ptwas)

        # match the gene names 
        sub_fastenloc <- fastenloc %>%
            select(gene=Gene, GLCP) %>%
            mutate(gene = str_replace_all(gene, "-", "\\."))
        head(sub_fastenloc)

        ptwas_fastenloc <- sub_ptwas %>%
            inner_join(sub_fastenloc)
        head(ptwas_fastenloc)
        dim(ptwas_fastenloc)

        res <- intact(GLCP_vec=ptwas_fastenloc$GLCP,
                    z_vec=ptwas_fastenloc$zscore,
                    prior_fun=linear
        )
        head(res)
        summary(res)

        # combine results
        intact_res <- cbind(ptwas_fastenloc, intact_pip=res) %>%
            arrange(-intact_pip)
        head(intact_res)
        return(intact_res)
        
    }


        # load ptwas data
        ptwas <- read_table("${ptwas_file}")
        # load fastenloc GLCP data
        fastenloc <- read_table("${fastenloc_file}")

        res <- run_intact(ptwas, fastenloc)
        saveRDS(res, "${_output}")

